In [1]:
import pyLDAvis.gensim  # Library for interactive topic model visualization
import torch  # PyTorch library for deep learning and GPU acceleration
from torch.utils.data import DataLoader  # Provides an iterator over a dataset for efficient batch processing
from tqdm import tqdm  # Creates progress bars to visualize the progress of loops or tasks
from gensim.models import LdaModel  # Implements LDA for topic modeling using the Gensim library
from gensim.corpora import Dictionary  # Represents a collection of text documents as a bag-of-words corpus
from gensim.models import CoherenceModel  # Computes coherence scores for topic models

import pickle
import os  # Provides functions for interacting with the operating system, such as creating directories
import itertools  # Provides various functions for efficient iteration and combination of elements
import numpy as np  # Library for numerical computing in Python, used for array operations and calculations
from time import time, sleep # Measures the execution time of code snippets or functions
import pprint as pp  # Pretty-printing library, used here to format output in a readable way
import pandas as pd
import logging # Logging module for generating log messages
import sys # Provides access to some variables used or maintained by the interpreter and to functions that interact with the interpreter 
import shutil # High-level file operations such as copying and removal 
import zipfile # Provides tools to create, read, write, append, and list a ZIP file
from tqdm.notebook import tqdm  # Creates progress bars in Jupyter Notebook environment
import json
import random
import logging
import csv
from dask.distributed import as_completed

In [2]:
# Configure root logger level (this will affect all loggers unless overridden)
logging.getLogger().setLevel(logging.ERROR)

# Create a file handler that logs messages to a file.
file_handler = logging.FileHandler('C:/_harvester/dask-logs/dask_logs.log')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

# Get the logger for distributed.utils_perf and add the file handler.
perf_logger = logging.getLogger('distributed.utils_perf')
perf_logger.addHandler(file_handler)
perf_logger.setLevel(logging.INFO)  # Adjust this level as needed

# Remove all handlers associated with the root logger (including default StreamHandler)
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

In [3]:
# Dask dashboard throws deprecation warnings w.r.t. Bokeh
import warnings
from bokeh.util.deprecation import BokehDeprecationWarning

# Disable Bokeh deprecation warnings
warnings.filterwarnings("ignore", category=BokehDeprecationWarning)
# Filter out the specific warning message
warnings.filterwarnings("ignore", module="distributed.utils_perf")

In [4]:

# Define the range of number of topics for LDA and step size
START_TOPICS = 1
END_TOPICS = 2
STEP_SIZE = 1

# define the decade that is being modelled 
DECADE = '2010s'

# In the case of this machine, since it has an Intel Core i9 processor with 8 physical cores (16 threads with Hyper-Threading), 
# it would be appropriate to set the number of workers in Dask Distributed LocalCluster to 8 or slightly lower to allow some CPU 
# resources for other tasks running on your system.
CORES = 6
THREADS_PER_CORE = 1

# specify the number of passes for Gensim LdaModel
PASSES = 15

# specify the number of iterations
ITERATIONS = 50

# specify the chunk size for LdaModel object
CHUNKSIZE = 4000

# Number of documents to process per iteration
BATCH_SIZE = 100

In [5]:


# create folder structure
log_dir = f"C:/_harvester/data/lda-models/{DECADE}_html/log/"
model_dir = f"C:/_harvester/data/lda-models/2010s_html/train-eval-data/"
image_dir = f"C:/_harvester/data/lda-models/{DECADE}_html/visuals/"

# Check if the directories exist and contain data
if os.path.exists(log_dir) and os.path.exists(model_dir) and os.path.exists(image_dir):
    log_files = os.listdir(log_dir)
    model_files = os.listdir(model_dir)
    image_files = os.listdir(image_dir)

    # Check if the directories are not empty
    if log_files or model_files or image_files:
        # Find an available filename for the archive
        counter = 0
        while True:
            archive_file = f"C:/_harvester/data/lda-models/{DECADE}_html/archive{counter:04d}.zip"
            if not os.path.exists(archive_file):
                break
            counter += 1

        # Create the zip file for archiving existing folders
        with zipfile.ZipFile(archive_file, 'w') as zipf:
            # Add log files to the zip file
            for log_file in log_files:
                zipf.write(os.path.join(log_dir, log_file), arcname=os.path.join("log", log_file))
            
            # Add model files to the zip file
            for model_file in model_files:
                zipf.write(os.path.join(model_dir, model_file), arcname=os.path.join("model", model_file))
            
            # Add image files to the zip file
            for image_file in image_files:
                zipf.write(os.path.join(image_dir, image_file), arcname=os.path.join("image", image_file))

        # Remove existing subdirectories after archiving them
        for subdir in [log_dir, model_dir, image_dir]:
            if os.path.exists(subdir):
                subfiles = os.listdir(subdir)
                for subfile in subfiles:
                    filepath = os.path.join(subdir, subfile)
                    if os.path.isdir(filepath):
                        os.rmdir(filepath)

# Create fresh directories for the new run
os.makedirs(log_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)
os.makedirs(image_dir, exist_ok=True)


In [6]:


# The parameter `alpha` in Latent Dirichlet Allocation (LDA) represents the concentration parameter of the Dirichlet 
# prior distribution for the topic-document distribution.
# It controls the sparsity of the resulting document-topic distributions.

# A lower value of `alpha` leads to sparser distributions, meaning that each document is likely to be associated with fewer topics.
# Conversely, a higher value of `alpha` encourages documents to be associated with more topics, resulting in denser distributions.

# The choice of `alpha` affects the balance between topic diversity and document specificity in LDA modeling.
alpha_values = np.arange(0.01, 1, 0.3).tolist()
alpha_values += ['symmetric', 'asymmetric']

# In Latent Dirichlet Allocation (LDA) topic analysis, the beta parameter represents the concentration 
# parameter of the Dirichlet distribution used to model the topic-word distribution. It controls the 
# sparsity of topics by influencing how likely a given word is to be assigned to a particular topic.

# A higher value of beta encourages topics to have a more uniform distribution over words, resulting in more 
# general and diverse topics. Conversely, a lower value of beta promotes sparser topics with fewer dominant words.

# The choice of beta can impact the interpretability and granularity of the discovered topics in LDA.
beta_values = np.arange(0.01, 1, 0.3).tolist()
beta_values += ['symmetric']


In [7]:

"""
The data_generator function is defined as a generator. It opens the specified JSON file (filename) 
and iterates over its lines using a for loop. Each line is parsed using json.loads() to convert it 
into a Python object (e.g., dictionary). The yield keyword is used instead of return to create a 
generator that produces one parsed JSON object at a time.

The num_samples variable counts the total number of lines in the JSON file by opening it (open(filename)) 
and iterating over its lines using a generator expression (sum(1 for _ in open(filename))). This gives 
us an estimate of how many samples are present in the dataset.

The num_train_samples variable calculates the desired number of samples for training based on the provided 
train_ratio. It multiplies num_samples by train_ratio, converting it to an integer using int().

Two empty lists, train_data and eval_data, are initialized to store training and evaluation datasets, respectively.

An instance of the `data_generator

"""

def futures_create_lda_datasets(filename, train_ratio):
    # Get the file size in bytes
    file_size = os.path.getsize(filename)

    # Get the last modified timestamp of the file
    last_modified = os.path.getmtime(filename)

    # Print the metadata
    print("\nFile Metadata:")
    print(f"Filename: {filename}")
    print(f"Size: {file_size} bytes")
    print(f"Last Modified: {last_modified}\n")
    
    with open(filename, 'r') as jsonfile:
        data = json.load(jsonfile)
    
    num_samples = len(data)  # Count the total number of samples
    num_train_samples = int(num_samples * train_ratio)  # Calculate the number of samples for training
    
    # Shuffle the data
    random.shuffle(data)

    train_data = data[:num_train_samples]  # Assign a portion of data for training
    eval_data = data[num_train_samples:]  # Assign the remaining data for evaluation

    print(f"Number of training samples: {len(train_data)}")
    print(f"Number of eval samples: {len(eval_data)}")

    # Create delayed objects for train and eval datasets
    future_train_data = dask.delayed(train_data)
    future_eval_data = dask.delayed(eval_data)

    return future_train_data, future_eval_data


In [8]:

"""
This method trains a Latent Dirichlet Allocation (LDA) model using the Gensim library. Here is a breakdown of the steps involved:

    (1)The method takes in parameters such as the number of topics (n_topics), alpha and beta hyperparameters, data (a list of documents), 
        and train_eval (a boolean indicating whether it's training or evaluation).

    (2)If train_eval is True, a logging configuration is set up to log training information to a file named "train-model.log". 
        Otherwise, it logs to "eval-model.log".

    (3) Two empty lists, combined_corpus and combined_text, are initialized to store the combined corpus and text.

    (4) The number of passes for training the LDA model is set to 11.

    (5) A loop iterates over each document in the data list. Inside the loop:
            - A Gensim Dictionary object is created from the current document.
            - The document is converted into a bag-of-words representation using doc2bow().
            - A PerplexityMetric object is created to track perplexity during training.
            - If combined_text is empty, indicating that it's the first iteration:
                The initial LDA model is trained using LdaModel() with parameters such as corpus, 
                id2word (the dictionary), num_topics, alpha, beta, random_state, passes, iterations, chunksize, and per_word_topics.

            - Otherwise:
                The existing LDA model is updated with new data using lda_model_gensim.update(corpus).
                The current document's text and corpus are added to combined_text and combined_corpus respectively.

    (6) Logging is shut down.

    (7) Finally, the trained LDA model (lda_model_gensim), combined_corpus, and combined_text are returned.
"""

def train_model(n_topics: int, alpha: list, beta: list, data: list, chunksize=BATCH_SIZE):
    combined_corpus = []  # Initialize list to store combined corpus
    combined_text = []
    
    # Convert the Delayed object to a Dask Bag and compute it to get the actual data
    streaming_documents = data.compute()
    
    # Load or create a dictionary outside the loop to track word IDs across batches
    dictionary_global = Dictionary()

    num_documents = len(streaming_documents)
    
    model_data = {
        'lda_model': [], # lda_model_gensim,
        'corpus': [], # combined_corpus,
        'text': [], #combined_text,
        'convergence': [], #convergence_score,
        'perplexity': [], # perplexity_score,
        'coherence': [], #coherence_score,
        'dictionary': [], #dictionary_global,
        'topics': [], #n_topics,
        'alpha': [], #alpha,
        'beta': []  #beta ,
    }

    batch_size = chunksize  # Number of documents to process per iteration
    
    for start_index in range(0, num_documents, batch_size):
        end_index = min(start_index + batch_size, num_documents)
        
        batch_documents = streaming_documents[start_index:end_index]

        for texts_out in batch_documents:
            dictionary_global.add_documents([texts_out])
            corpus_single_doc = [dictionary_global.doc2bow(texts_out)]
            
            lda_model_gensim = LdaModel(corpus=corpus_single_doc,
                                        id2word=dictionary_global,
                                        num_topics=n_topics,
                                        alpha=alpha,
                                        eta=beta,
                                        random_state=75,
                                        passes=PASSES,
                                        iterations=ITERATIONS,
                                        chunksize=CHUNKSIZE,
                                        per_word_topics=True)

            combined_text.extend(texts_out)
            combined_corpus.extend(corpus_single_doc)

            # calculate metrics
            convergence_score = None
            perplexity_score = None
            coherence_score = None

            convergence_score = lda_model_gensim.bound(corpus_single_doc)
            perplexity_score = lda_model_gensim.log_perplexity(corpus_single_doc)

            # Check for runtime warnings during topic coherence calculation
            with np.errstate(divide='ignore', invalid='ignore'):
                coherence_model = CoherenceModel(model=lda_model_gensim, texts=texts_out, dictionary=dictionary_global, coherence='c_v')
                coherence_score = coherence_model.get_coherence()

            model_data['lda_model'].append(lda_model_gensim)
            model_data['corpus'].append(combined_corpus)
            model_data['text'].append(combined_text)
            model_data['convergence'].append(convergence_score)
            model_data['perplexity'].append(perplexity_score)
            model_data['coherence'].append(coherence_score)
            model_data['dictionary'].append(dictionary_global)
            model_data['topics'].append(n_topics)
            model_data['alpha'].append(alpha)
            model_data['beta'].append(beta)

    # Verify that combined_text contains all the original text
    original_tokens = sum((len(doc) for doc in streaming_documents), 0)
    assert len(combined_text) == original_tokens, "Combined text does not contain all the original text"

    return (lda_model_gensim, \
            n_topics, \
            alpha, \
            beta, \
            model_data,)

In [9]:
#%%
# Define a function to save models and log data
def save_model_and_log(model_data, n_topics, alpha, beta, lda_model, model_dir, log_dir, train_or_eval=None):
    # Normalize alpha and beta values into strings suitable for filenames
    alpha_str = '_'.join(map(str, alpha)) if isinstance(alpha, list) else str(alpha)
    beta_str = '_'.join(map(str, beta)) if isinstance(beta, list) else str(beta)

    # Construct a unique filename for each model using its parameters
    if train_or_eval == True:
        filename = f"train-lda_model_topics{n_topics}_alpha{alpha_str}_beta{beta_str}.model"
    else:
        filename = f"eval-lda_model_topics{n_topics}_alpha{alpha_str}_beta{beta_str}.model"
    
    # Ensure that any special characters are removed or replaced in filename components
    filename = filename.replace('.', 'p').replace('/', '-').replace('\\', '-')
    
    filepath = os.path.join(model_dir, filename)
    
    # Save the model
    lda_model.save(filepath)

    # Specify the filename for the CSV file
    if train_or_eval == True:
        csv_filename = "train-lda-model-train-data.csv"
    else:
        csv_filename = 'eval-lda-model-train-data.csv'
        
    csv_path = os.path.join(log_dir, csv_filename)

    # Check if the CSV file exists
    file_exists = os.path.isfile(csv_path)

    # Write or append data to the CSV file
    with open(csv_path, mode='a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        # Write header row if file doesn't exist
        if not file_exists:
            writer.writerow(model_data.keys())
        
        # Append data rows
        writer.writerow(model_data.values())

In [10]:

if __name__=="__main__":
    import dask   # Parallel computing library that scales Python workflows across multiple cores or machines 
    from dask.distributed import Client, LocalCluster, wait   # Distributed computing framework that extends Dask functionality 
    from dask.diagnostics import ProgressBar   # Visualizes progress of Dask computations
    from dask.distributed import progress
    from dask.delayed import Delayed # Decorator for creating delayed objects in Dask computations
    from dask.distributed import as_completed
    from dask.bag import Bag
    from dask import delayed
    import dask.config
    from dask.distributed import wait

    # Specify the local directory path
    DASK_DIR = '/_harvester/tmp-dask-out'

    # Deploy a Single-Machine Multi-GPU Cluster
    # https://medium.com/@aryan.gupta18/end-to-end-recommender-systems-with-merlin-part-1-89fabe2fa05b
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Specify GPU device IDs
    protocol = "tcp"  # "tcp" or "ucx"
    num_gpus = 1
    NUM_GPUS=[0]
    visible_devices = ",".join([str(n) for n in NUM_GPUS])  # Select devices to place workers
    device_limit_frac = 0.7  # Spill GPU-Worker memory to host at this limit.
    device_pool_frac = 0.8
    part_mem_frac = 0.15

    # Manually specify the total device memory size (in bytes)
    device_size = 10 * 1024 * 1024 * 1024  # GPU has 12GB but setting at 10GB
            
    ram_memory_limit = "100GB" # Set the RAM memory limit (per worker)
    device_limit = int(device_limit_frac * device_size)
    device_pool_size = int(device_pool_frac * device_size)
    part_size = int(part_mem_frac * device_size)

    cluster = LocalCluster(
            n_workers=CORES,
            threads_per_worker=THREADS_PER_CORE,
            processes=False,
            memory_limit=ram_memory_limit,
            local_directory=DASK_DIR,
            dashboard_address=None,
            #dashboard_address=":8787",
            #protocol="tcp",
    )


    # Create the distributed client
    client = Client(cluster)

    # Get information about workers from scheduler
    workers_info = client.scheduler_info()["workers"]

    # Iterate over workers and set their memory limits
    for worker_id, worker_info in workers_info.items():
        worker_info["memory_limit"] = ram_memory_limit

    # Verify that memory limits have been set correctly
    #for worker_id, worker_info in workers_info.items():
    #    print(f"Worker {worker_id}: Memory Limit - {worker_info['memory_limit']}")

    # Check if the Dask client is connected to a scheduler:
    if client.status == "running":
        print("Dask client is connected to a scheduler.")
        # Scatter the embedding vectors across Dask workers
    else:
        print("Dask client is not connected to a scheduler.")

    # Check if Dask workers are running:
    if len(client.scheduler_info()["workers"]) > 0:
        print("Dask workers are running.")
    else:
        print("No Dask workers are running.")

    
    # Load data from the JSON file
    filename = "C:/_harvester/data/tokenized-sentences/10s/tokenized_sents-w-bigrams.json"
    train_ratio = 0.8
    

    # create training and evaluation data
    print("Creating training and evaluation samples...")
    started = time()
    future = client.submit(futures_create_lda_datasets, filename, train_ratio)
    print(f"Completed creation of training and evaluation samples in {round((time()- started)/60,2)} minutes.\n")

    # Wait for future to complete and retrieve results
    train_data, eval_data = future.result()

    # Scatter the computed training and evaluation data across workers
    print("Beginning data scatter...")
    scattered_train_data_future = client.scatter(train_data)
    scattered_eval_data_future = client.scatter(eval_data)
    print("Data scatter complete...\n")
    

    train_futures = []  # List to store futures for training
    eval_futures = []  # List to store futures for evaluation


# Loop over different values of n_topics, alpha_value, and beta_value
for n_topics in range(START_TOPICS, END_TOPICS + 1, STEP_SIZE):
    for alpha_value in alpha_values:
        for beta_value in beta_values:
            #print(f"Training model with n_topics={n_topics}, alpha={alpha_value}, beta={beta_value}")

            # Submit train_model function directly with scattered futures
            future_train = client.submit(train_model, n_topics, alpha_value, beta_value, scattered_train_data_future)
            future_eval = client.submit(train_model, n_topics, alpha_value, beta_value, scattered_eval_data_future)

            # Training models
            def callback_train(future):
                try:
                    # Retrieve the result of the training future
                    result_train = future.result()
                except Exception as e:
                    print(f"Error occurred during training: {e}")
                else:
                    # Save the trained model and log data
                    save_model_and_log(*result_train[1:], model_dir=model_dir,
                                    log_dir=log_dir, train_or_eval=True)

            # Evaluation models
            def callback_eval(future):
                try:
                    # Retrieve the result of the evaluation future
                    result_eval = future.result()
                except Exception as e:
                    print(f"Error occurred during evaluation: {e}")
                else:
                    # Save the evaluation model and log data
                    save_model_and_log(*result_eval[1:], model_dir=model_dir,
                                    log_dir=log_dir, train_or_eval=False)


            # Add a callback to save the model once training is complete
            future_train.add_done_callback(callback_train)
            future_eval.add_done_callback(callback_eval)

            # Append futures to separate lists for training and evaluation models
            train_futures.append(future_train)
            eval_futures.append(future_eval)

    print("Model training and evaluation tasks submitted...\n")

    # Now, instead of waiting for all futures to complete with wait(),
    # we can process them as they complete using as_completed.
    #for future in as_completed(train_futures + eval_futures):
    for future in as_completed(train_futures + eval_futures):
        # As each future completes, its result will be processed by the callback function.
        pass  # The 'pass' statement is just a placeholder since the actual work is done in callbacks.

    # Gather all trained models back to the client (optional)
    print("gathering all trained models back to the client...")
    trained_models = [future.result() for future in train_futures]
    evaluation_models = [future.result() for future in eval_futures]

    # Save each trained model with specific filenames based on its parameters outside of the loop
    #print("Saving models...")
    for model_data in tqdm(trained_models, desc="Saving training models...", total=len(trained_models)):
        model_data, n_topics, alpha_value, beta_value,
        lda_model = model_data[0]  # Assuming the trained model is the last element in the result tuple
        save_model_and_log(model_data, n_topics, alpha_value, beta_value, lda_model, model_dir=model_dir, log_dir=log_dir, train_or_eval=True)

    # Save each trained model with specific filenames based on its parameters outside of the loop
    #print("Saving models...")
    for model_data in tqdm(evaluation_models, desc="Saving evaluation models...", total=len(evaluation_models)):
        model_data, n_topics, alpha_value, beta_value,
        lda_model = model_data[0]  # Assuming the trained model is the last element in the result tuple
        save_model_and_log(model_data, n_topics, alpha_value, beta_value, lda_model, model_dir=model_dir, log_dir=log_dir, train_or_eval=False)

    print("Completed saving models and logging data.")

    # Close Dask client after all operations are complete
    client.close()

Dask client is connected to a scheduler.
Dask workers are running.
Creating training and evaluation samples...
Completed creation of training and evaluation samples in 0.0 minutes.


File Metadata:
Filename: C:/_harvester/data/tokenized-sentences/10s/tokenized_sents-w-bigrams.json
Size: 81447561 bytes
Last Modified: 1713177730.7095945

Number of training samples: 333967
Number of eval samples: 83492
Beginning data scatter...
Data scatter complete...

Model training and evaluation tasks submitted...



c:\Users\pqn7\.conda\envs\nlp\lib\site-packages\distributed\client.py:3157: UserWarning: Sending large graph of size 14.38 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\pqn7\.conda\envs\nlp\lib\site-packages\distributed\client.py:3157: UserWarning: Sending large graph of size 57.34 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
2024-04-18 08:49:51,800 - distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
2024-04-18 08:49:59,494 - distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
2024-04-18 08:50:26,403 - distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
2024-04-18 08:50:41,110 - distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
2024-04-18 08:

In [ ]:
logging.shutdown()